In [1]:
data = '../data/2022_07_13_WTvsKO_allcompounds_neg.csv'

a = 5
b = 11

D = 1.0063

m1 = a*D
m2 = b*D

In [2]:
import pandas as pd
import numpy as np

### fcn to upload data

In [3]:
def upload(file):
    df = pd.read_csv(file, header=2) #header=2 cuts off first two rows
    return df

df_raw = upload(data)

- sort df at beginning by m/z
- define m1 and m2 at beginning; applicable to pair picking as well as mass adjustment
- m1 = a*1.0063, m2 = b*1.0063
- input for number of compounds being analyzed, rather than finding the '.1' values

### fcn to format data

In [4]:
def format(df):
    col = df.columns.tolist() #create a list of all column names
    main = [0,2,4,5] #Index of main columns we wish to keep and compare
    stop = col.index([col for col in df.columns if '.1' in col][0]) #index of duplicate columns we don't need
    intensities = col[16:stop] #intensity columns we wish to keep
    col_main = [] #Column names of kept columns
    for i in main:
        col_main.append(col[i])
    '''Create new filtered dataframe of important columns'''
    df_keep = df[col_main + intensities] #.sort_values(["m/z"], ascending=False)
    return df_keep

df_keep = format(df_raw)[:100]

df_keep

,Compound,m/z,Retention time (min),CCS (angstrom^2),NEG_2022_07_12_NR_LIPIDS_HILIC_KO_C1,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_C2,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_D1,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_D2,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_R1,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_R2,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_RD1,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_RD2,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_C1,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_C2,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_D1,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_D2,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_R1,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_R2,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_RD1,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_RD2
0,5.09_803.7568m/z,803.756835,5.086917,284.123327,0.000000,0.000000,407.654500,420.461276,0.000000,0.000000,385.107511,365.595946,0.000000,0.000000,449.254529,453.605111,0.000000,0.000000,423.162041,420.887528
1,4.97_787.7598m/z,787.759812,4.965617,281.068778,0.000000,0.000000,369.250626,370.267982,0.000000,0.000000,313.076980,303.813405,0.000000,0.000000,257.533814,262.038949,0.000000,0.000000,226.896927,250.294353
2,5.05_757.2737m/z,757.273727,5.053083,278.108306,0.000000,0.000000,307.498293,286.800966,0.000000,0.000000,283.482355,293.671320,0.000000,0.000000,182.472212,195.835673,0.000000,0.000000,182.637573,174.141373
3,6.29_1020.8231m/z,1020.823084,6.294300,323.846316,0.000000,0.000000,129.718470,118.280514,0.000000,0.000000,127.239392,127.443142,0.000000,0.000000,188.296237,196.759510,0.000000,0.000000,162.551127,163.469014
4,5.09_776.6017n,797.588528,5.086917,284.160322,3387.908900,3405.940996,24694.959150,25348.674536,3275.245776,3327.040540,24091.229533,23828.255412,2856.702211,2874.484875,26770.898125,26762.333436,2593.152599,2638.692648,25546.983784,25205.844765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.62_369.3644n,390.340211,0.620983,202.255685,1618.208588,1622.321749,6856.139386,6399.555773,1521.636412,1421.029030,6568.122120,6243.677971,1455.379774,1381.153808,6594.662870,6298.677456,1476.623952,1554.881081,5857.758896,5819.999682
96,0.78_843.6165n,878.585879,0.776150,293.149110,1432.997409,1262.329287,8931.584877,8766.189473,1252.210412,1347.945276,8079.402351,8602.881088,1471.031442,1571.597336,11885.916450,12508.657713,1379.010910,1455.039994,10709.539311,11117.655476
97,4.31_585.8916n,606.879107,4.311150,196.631411,104.443668,105.566408,189.556600,188.351481,84.361515,85.789276,166.906520,171.013502,130.046156,136.844423,235.341009,242.168060,112.919553,114.702008,227.704351,226.749327
98,3.16_898.3094m/z,898.309447,3.157350,302.473096,0.000000,0.000000,1438.473443,1416.179599,0.000000,0.009136,1291.361915,1471.353534,0.104422,0.928731,1203.505933,1410.942499,0.000000,0.000000,1223.720464,1305.717719


### fcn to pick out pairs

In [5]:
def pick_pairs(df):
    '''Returns index pairs of compounds with same RT and CCS'''

    '''Define lists and tolerances of each column to compare with itself'''
    mz = df['m/z']
    mz_tol = 1e-4
    rt = df['Retention time (min)']
    rt_tol = 1e-3
    ccs = df['CCS (angstrom^2)']
    ccs_tol = 1e-3

    '''Initialize a list for indexes to be held for each pair of matched values'''
    idxs = []
    '''Create nested for loop to compare i with j in each column'''
    for i in range(len(df)):
        for j in range(len(df)):
            '''Define checks for each column'''
            check_mz = np.isclose(mz[i], mz[j] + (m2 - m1), mz_tol)        
            check_rt = np.isclose(rt[i], rt[j], rt_tol)
            check_ccs = np.isclose(ccs[i], ccs[j], ccs_tol)
            
            '''Record results of each check (True/False)'''
            checks = [check_mz, check_rt, check_ccs]
            
            '''If all checks are true, append to list of pairs'''
            if all(checks) and i!=j:
                idxs.append([i,j])
                pairs = np.array(idxs)
            else:
                pass

    return pairs

In [6]:
pairs = pick_pairs(df_keep)
pairs

array([[14, 45],
       [16, 15],
       [20, 18],
       [28, 24],
       [36, 24],
       [38, 26],
       [44, 10],
       [54, 75],
       [56, 30],
       [70,  4],
       [92, 90]])

### Show dataframe of pairs

In [7]:
df_pairs = df_keep.iloc[pairs.flatten()]
df_pairs.head()

,Compound,m/z,Retention time (min),CCS (angstrom^2),NEG_2022_07_12_NR_LIPIDS_HILIC_KO_C1,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_C2,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_D1,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_D2,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_R1,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_R2,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_RD1,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_RD2,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_C1,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_C2,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_D1,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_D2,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_R1,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_R2,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_RD1,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_RD2
14,5.05_761.6121m/z,761.612050,5.053083,278.080052,344.694030,346.519756,29914.321156,30126.459092,327.583545,322.763704,28831.778489,28995.641196,566.946809,497.744051,23269.250204,24470.166661,511.791139,503.041123,23047.021804,22961.231380
45,5.05_756.6069n,755.566165,5.053083,278.119515,10104.471419,9758.085041,40949.186367,41034.038834,9463.885209,9806.965957,39326.224454,39696.824424,9378.739955,9352.862476,32623.954491,34517.143737,8604.682340,8469.671476,32522.262396,32620.870473
16,3.16_899.8980m/z,899.897980,3.157350,302.465024,0.000000,0.000000,320.298388,291.122259,0.000000,0.000000,288.235085,321.229091,0.000000,0.000000,232.314163,277.366148,0.000000,0.000000,242.553709,249.178319
15,3.16_893.9342m/z,893.934249,3.157350,302.495477,0.000000,0.000000,255.139101,252.438602,0.000000,0.000000,248.483765,293.242786,0.000000,0.000000,209.950323,237.755500,0.000000,0.000000,203.491538,214.805597
20,5.05_342.4169m/z,342.416944,5.053083,190.202882,0.000000,0.000000,37.936238,38.278801,0.000000,0.000000,35.389910,31.514619,0.000000,0.000000,28.595740,31.986394,0.000000,0.000000,25.260168,24.858283


### fcn to adjust masses of each pair

In [9]:
def mass_adj(pairs, df, x, y):
    '''Adjusts masses of given dataframe and list of pairs. Pairs must be together, with higher mass first. x is the lower value, y is the higher value.'''
    
    df_pairs = df.iloc[pairs.flatten()]
    masses = np.array(df_pairs["m/z"]).reshape((len(pairs), 2))
    masses[:, 0] -= y
    masses[:, 1] -= x

    df_pairs.insert(2, "m/z_adj", masses.flatten().tolist())

    return df_pairs

df_adj = mass_adj(pairs, df_keep, m1, m2)
df_adj

,Compound,m/z,m/z_adj,Retention time (min),CCS (angstrom^2),NEG_2022_07_12_NR_LIPIDS_HILIC_KO_C1,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_C2,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_D1,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_D2,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_R1,...,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_RD1,NEG_2022_07_12_NR_LIPIDS_HILIC_KO_RD2,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_C1,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_C2,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_D1,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_D2,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_R1,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_R2,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_RD1,NEG_2022_07_12_NR_LIPIDS_HILIC_WT_RD2
14,5.05_761.6121m/z,761.612050,750.542750,5.053083,278.080052,344.694030,346.519756,29914.321156,30126.459092,327.583545,...,28831.778489,28995.641196,566.946809,497.744051,23269.250204,24470.166661,511.791139,503.041123,23047.021804,22961.231380
45,5.05_756.6069n,755.566165,750.534665,5.053083,278.119515,10104.471419,9758.085041,40949.186367,41034.038834,9463.885209,...,39326.224454,39696.824424,9378.739955,9352.862476,32623.954491,34517.143737,8604.682340,8469.671476,32522.262396,32620.870473
16,3.16_899.8980m/z,899.897980,888.828680,3.157350,302.465024,0.000000,0.000000,320.298388,291.122259,0.000000,...,288.235085,321.229091,0.000000,0.000000,232.314163,277.366148,0.000000,0.000000,242.553709,249.178319
15,3.16_893.9342m/z,893.934249,888.902749,3.157350,302.495477,0.000000,0.000000,255.139101,252.438602,0.000000,...,248.483765,293.242786,0.000000,0.000000,209.950323,237.755500,0.000000,0.000000,203.491538,214.805597
20,5.05_342.4169m/z,342.416944,331.347644,5.053083,190.202882,0.000000,0.000000,37.936238,38.278801,0.000000,...,35.389910,31.514619,0.000000,0.000000,28.595740,31.986394,0.000000,0.000000,25.260168,24.858283
18,5.05_336.3790m/z,336.378983,331.347483,5.053083,190.331918,0.000000,0.000000,49.758595,50.075138,0.000000,...,43.173579,46.655149,0.000000,0.000000,33.143198,37.191122,0.000000,0.000000,27.293195,29.985606
28,0.93_895.0867m/z,895.086674,884.017374,0.931300,293.065615,0.000000,0.000000,670.308318,710.602943,0.000000,...,547.286806,698.311093,0.000000,0.000000,568.188898,701.502070,0.000000,0.000000,529.858379,536.376848
24,0.93_889.0476m/z,889.047570,884.016070,0.931300,293.095816,0.000000,0.084834,791.377475,857.835133,0.000000,...,706.702622,913.192149,0.000000,0.000000,750.402776,835.898574,0.000000,0.000000,646.943583,692.763594
36,0.93_895.1727m/z,895.172742,884.103442,0.931300,293.065187,0.000000,0.000000,828.728325,821.198530,0.000000,...,624.682320,845.797032,0.000000,0.000000,681.199215,825.912194,0.000000,0.000000,544.577782,606.057183
24,0.93_889.0476m/z,889.047570,884.016070,0.931300,293.095816,0.000000,0.084834,791.377475,857.835133,0.000000,...,706.702622,913.192149,0.000000,0.000000,750.402776,835.898574,0.000000,0.000000,646.943583,692.763594
